In [ ]:
!pip install openai requests beautifulsoup4 -q



In [ ]:
import json
import time
import random
import re
from datetime import datetime
from typing import Dict, List, Any


import requests
from bs4 import BeautifulSoup


In [ ]:
class BaseAgent:
    """
    Base class for all AI agents in our system.
    Each agent will have a name, role, and the ability to process tasks.
    """

    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role
        self.history = []  # Keep track of what this agent has done

    def log_activity(self, activity: str):
        """Log what the agent is doing"""
        timestamp = datetime.now().strftime('%H:%M:%S')
        log_entry = f"[{timestamp}] {self.name}: {activity}"
        self.history.append(log_entry)
        print(f" {log_entry}")

    def process_task(self, task_data: Dict[str, Any]) -> Dict[str, Any]:
        """Base method that all agents will override"""
        raise NotImplementedError("Each agent must implement process_task method")


test_agent = BaseAgent("TestBot", "Testing")
test_agent.log_activity("Base agent class created successfully!")
print(f" Agent '{test_agent.name}' with role '{test_agent.role}' is ready!")

 [04:30:02] TestBot: Base agent class created successfully!
 Agent 'TestBot' with role 'Testing' is ready!


In [ ]:
class ResearchAgent(BaseAgent):
    """
    Research Agent: Specializes in gathering information about topics
    In a real system, this would search the web, databases, or APIs
    """

    def __init__(self):
        super().__init__("Research Agent", "Information Gathering")
        # Simulated knowledge base (in real app, this would be web search results)
        self.knowledge_base = {
            "artificial intelligence": [
                "AI is transforming industries worldwide",
                "Machine learning is a subset of AI that enables computers to learn",
                "Deep learning uses neural networks with multiple layers",
                "AI applications include healthcare, finance, transportation, and education",
                "Ethical AI development is crucial for responsible technology advancement"
            ],
            "climate change": [
                "Global temperatures have risen by 1.1°C since pre-industrial times",
                "Renewable energy adoption is accelerating worldwide",
                "Carbon emissions must be reduced by 45% by 2030",
                "Climate change affects weather patterns, ecosystems, and agriculture",
                "International cooperation is essential for climate action"
            ],
            "space exploration": [
                "Private companies are revolutionizing space travel",
                "Mars exploration missions are planned for the 2030s",
                "Space telescopes are discovering thousands of exoplanets",
                "Asteroid mining could provide valuable resources",
                "International space cooperation continues with ISS and beyond"
            ]
        }

    def simulate_web_search(self, topic: str) -> List[str]:
        """Simulate searching the web for information"""
        self.log_activity(f"Searching for information about: {topic}")


        time.sleep(1)


        topic_lower = topic.lower()
        results = []


        for key, facts in self.knowledge_base.items():
            if key in topic_lower or any(word in topic_lower for word in key.split()):
                results.extend(facts)


        if not results:
            results = [
                f"Research shows that {topic} is an important and evolving field",
                f"Multiple perspectives exist on {topic} in current literature",
                f"Recent developments in {topic} show promising trends",
                f"Experts recommend further investigation into {topic}",
                f"The impact of {topic} continues to grow in various sectors"
            ]

        self.log_activity(f"Found {len(results)} relevant information pieces")
        return results

    def process_task(self, task_data: Dict[str, Any]) -> Dict[str, Any]:
        """Main research processing method"""
        topic = task_data.get('topic', 'General Topic')

        self.log_activity(f"Starting research on: {topic}")


        research_results = self.simulate_web_search(topic)


        research_output = {
            'agent': self.name,
            'topic': topic,
            'research_completed': True,
            'findings': research_results,
            'sources_count': len(research_results),
            'research_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        self.log_activity(f"Research completed! Found {len(research_results)} key findings")

        return research_output


print(" Testing Research Agent...")
research_agent = ResearchAgent()
test_research = research_agent.process_task({'topic': 'Artificial Intelligence'})
print(f" Research Agent created and tested successfully!")
print(f" Sample findings: {len(test_research['findings'])} pieces of information gathered")

 Testing Research Agent...
 [04:31:48] Research Agent: Starting research on: Artificial Intelligence
 [04:31:48] Research Agent: Searching for information about: Artificial Intelligence
 [04:31:49] Research Agent: Found 5 relevant information pieces
 [04:31:49] Research Agent: Research completed! Found 5 key findings
 Research Agent created and tested successfully!
 Sample findings: 5 pieces of information gathered


In [ ]:
class WritingAgent(BaseAgent):
    """
    Writing Agent: Specializes in creating engaging content from research data
    This agent transforms raw information into readable blog posts
    """

    def __init__(self):
        super().__init__("Writing Agent", "Content Creation")
        self.writing_styles = {
            'engaging': {
                'intro_phrases': [
                    "Have you ever wondered about",
                    "In today's rapidly evolving world",
                    "Imagine a future where",
                    "What if I told you that"
                ],
                'transition_phrases': [
                    "Furthermore,", "Additionally,", "Moreover,",
                    "On the other hand,", "What's more,"
                ]
            }
        }

    def create_engaging_intro(self, topic: str) -> str:
        """Generate an engaging introduction"""
        intro_templates = [
            f"Have you ever wondered what the future holds for {topic}? You're not alone.",
            f"In today's rapidly changing world, {topic} is becoming increasingly important.",
            f"What if I told you that {topic} could revolutionize the way we live and work?",
            f"Imagine a world where {topic} plays a central role in our daily lives."
        ]
        return random.choice(intro_templates)

    def structure_content(self, research_data: Dict[str, Any]) -> str:
        """Create a well-structured blog post from research findings"""
        topic = research_data['topic']
        findings = research_data['findings']

        self.log_activity(f"Structuring content for: {topic}")


        blog_post = []


        blog_post.append(f"# The Future of {topic.title()}: What You Need to Know")
        blog_post.append("")


        intro = self.create_engaging_intro(topic)
        blog_post.append(intro)
        blog_post.append("")


        blog_post.append(f"## Understanding {topic.title()}")
        blog_post.append("")


        for i, finding in enumerate(findings[:3]):  # Use first 3 findings for main content
            if i > 0:
                blog_post.append("")  # Add spacing between paragraphs


            expanded_paragraph = self.expand_finding(finding, topic)
            blog_post.append(expanded_paragraph)


        if len(findings) > 3:
            blog_post.append("")
            blog_post.append(f"## Key Developments in {topic.title()}")
            blog_post.append("")

            for finding in findings[3:5]:  # Use next 2 findings
                expanded_paragraph = self.expand_finding(finding, topic)
                blog_post.append(expanded_paragraph)
                blog_post.append("")


        blog_post.append("## Looking Forward")
        blog_post.append("")
        conclusion = f"As we've explored, {topic} represents a fascinating area of development with significant implications for our future. The key is to stay informed and engaged with these evolving trends."
        blog_post.append(conclusion)

        return "\n".join(blog_post)

    def expand_finding(self, finding: str, topic: str) -> str:
        """Expand a research finding into a full paragraph"""
        # Add context and elaboration to make findings more readable
        expansions = [
            f"{finding} This development has significant implications for how we approach {topic} in the coming years.",
            f"Research indicates that {finding.lower()} The impact of this trend extends beyond immediate applications.",
            f"One of the most important aspects to consider is that {finding.lower()} This insight helps us understand the broader landscape.",
            f"Experts have noted that {finding.lower()} This perspective offers valuable guidance for future planning."
        ]

        return random.choice(expansions)

    def process_task(self, task_data: Dict[str, Any]) -> Dict[str, Any]:
        """Main writing processing method"""
        research_data = task_data.get('research_data', {})

        if not research_data:
            self.log_activity("No research data provided!")
            return {'error': 'No research data available for writing'}

        topic = research_data.get('topic', 'Unknown Topic')
        self.log_activity(f"Starting to write blog post about: {topic}")


        blog_content = self.structure_content(research_data)


        word_count = len(blog_content.split())
        paragraph_count = blog_content.count('\n\n')

        writing_output = {
            'agent': self.name,
            'topic': topic,
            'writing_completed': True,
            'blog_post': blog_content,
            'word_count': word_count,
            'paragraph_count': paragraph_count,
            'writing_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        self.log_activity(f"Blog post completed! {word_count} words, {paragraph_count} paragraphs")

        return writing_output


print(" Testing Writing Agent...")
writing_agent = WritingAgent()


test_writing = writing_agent.process_task({'research_data': test_research})
print(f" Writing Agent created and tested successfully!")
print(f" Blog post created: {test_writing['word_count']} words")

 Testing Writing Agent...
 [04:34:15] Writing Agent: Starting to write blog post about: Artificial Intelligence
 [04:34:15] Writing Agent: Structuring content for: Artificial Intelligence
 [04:34:15] Writing Agent: Blog post completed! 175 words, 10 paragraphs
 Writing Agent created and tested successfully!
 Blog post created: 175 words


In [ ]:
class ReviewingAgent(BaseAgent):
    """
    Reviewing Agent: Specializes in improving content quality
    This agent checks grammar, tone, flow, and provides suggestions
    """

    def __init__(self):
        super().__init__("Reviewing Agent", "Content Quality Improvement")
        self.improvement_categories = [
            'Grammar & Style',
            'Flow & Readability',
            'Engagement & Tone',
            'Structure & Organization',
            'Factual Consistency'
        ]

    def analyze_readability(self, text: str) -> Dict[str, Any]:
        """Analyze text readability and provide metrics"""
        sentences = text.split('.')
        words = text.split()
        paragraphs = text.split('\n\n')


        avg_words_per_sentence = len(words) / max(len(sentences), 1)
        avg_sentences_per_paragraph = len(sentences) / max(len(paragraphs), 1)

        readability_score = "Good"
        if avg_words_per_sentence > 25:
            readability_score = "Needs Improvement - Too Complex"
        elif avg_words_per_sentence < 10:
            readability_score = "Needs Improvement - Too Simple"

        return {
            'word_count': len(words),
            'sentence_count': len(sentences),
            'paragraph_count': len(paragraphs),
            'avg_words_per_sentence': round(avg_words_per_sentence, 1),
            'avg_sentences_per_paragraph': round(avg_sentences_per_paragraph, 1),
            'readability_score': readability_score
        }

    def generate_improvements(self, content: str, topic: str) -> List[Dict[str, str]]:
        """Generate specific improvement suggestions"""
        improvements = []


        if content.count('!') > 5:
            improvements.append({
                'category': 'Grammar & Style',
                'issue': 'Excessive exclamation marks',
                'suggestion': 'Reduce exclamation marks to maintain professional tone'
            })

        if 'very' in content.lower():
            improvements.append({
                'category': 'Grammar & Style',
                'issue': 'Weak intensifiers found',
                'suggestion': 'Replace "very" with more specific descriptive words'
            })


        paragraphs = [p for p in content.split('\n\n') if p.strip() and not p.startswith('#')]
        long_paragraphs = [p for p in paragraphs if len(p.split()) > 100]

        if long_paragraphs:
            improvements.append({
                'category': 'Flow & Readability',
                'issue': f'{len(long_paragraphs)} paragraphs are too long',
                'suggestion': 'Break down long paragraphs into smaller, more digestible chunks'
            })


        if not any(word in content.lower() for word in ['you', 'your', 'we', 'our']):
            improvements.append({
                'category': 'Engagement & Tone',
                'issue': 'Limited reader engagement',
                'suggestion': 'Add more direct address to readers using "you" and "your"'
            })


        improvements.append({
            'category': 'Structure & Organization',
            'issue': 'Content structure analysis',
            'suggestion': f'Consider adding more subheadings to break up content about {topic}'
        })

        improvements.append({
            'category': 'Engagement & Tone',
            'issue': 'Engagement opportunities',
            'suggestion': 'Consider adding a call-to-action or questions to encourage reader interaction'
        })

        return improvements

    def create_improved_version(self, original_content: str) -> str:
        """Create an improved version of the content"""
        improved_content = original_content


        improved_content = improved_content.replace(' very ', ' extremely ')
        improved_content = improved_content.replace(' Very ', ' Extremely ')


        if not improved_content.endswith('?'):
            improved_content += "\n\n**What are your thoughts on this topic? Share your perspective in the comments below!**"

        return improved_content

    def process_task(self, task_data: Dict[str, Any]) -> Dict[str, Any]:
        """Main reviewing processing method"""
        writing_data = task_data.get('writing_data', {})

        if not writing_data or 'blog_post' not in writing_data:
            self.log_activity("No writing data provided for review!")
            return {'error': 'No writing data available for review'}

        topic = writing_data.get('topic', 'Unknown Topic')
        original_content = writing_data['blog_post']

        self.log_activity(f"Starting review of blog post about: {topic}")


        readability_analysis = self.analyze_readability(original_content)
        improvements = self.generate_improvements(original_content, topic)
        improved_content = self.create_improved_version(original_content)


        original_word_count = len(original_content.split())
        improved_word_count = len(improved_content.split())

        review_output = {
            'agent': self.name,
            'topic': topic,
            'review_completed': True,
            'original_content': original_content,
            'improved_content': improved_content,
            'readability_analysis': readability_analysis,
            'improvement_suggestions': improvements,
            'original_word_count': original_word_count,
            'improved_word_count': improved_word_count,
            'review_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        self.log_activity(f"Review completed! {len(improvements)} suggestions provided")

        return review_output


print(" Testing Reviewing Agent...")
reviewing_agent = ReviewingAgent()


test_review = reviewing_agent.process_task({'writing_data': test_writing})
print(f" Reviewing Agent created and tested successfully!")
print(f" Review completed: {len(test_review['improvement_suggestions'])} suggestions provided")

 Testing Reviewing Agent...
 [04:36:38] Reviewing Agent: Starting review of blog post about: Artificial Intelligence
 [04:36:38] Reviewing Agent: Review completed! 2 suggestions provided
 Reviewing Agent created and tested successfully!
 Review completed: 2 suggestions provided


In [ ]:
class MultiAgentCoordinator:
    """
    Multi-Agent Coordinator: Manages the workflow between all agents
    This is the 'brain' that orchestrates the research → write → review process
    """

    def __init__(self):
        self.name = "Multi-Agent Coordinator"
        self.research_agent = ResearchAgent()
        self.writing_agent = WritingAgent()
        self.reviewing_agent = ReviewingAgent()
        self.workflow_history = []

    def log_workflow(self, step: str, details: str):
        """Log workflow progress"""
        timestamp = datetime.now().strftime('%H:%M:%S')
        log_entry = f"[{timestamp}]  {self.name}: {step} - {details}"
        self.workflow_history.append(log_entry)
        print(log_entry)

    def run_complete_workflow(self, topic: str) -> Dict[str, Any]:
        """
        Run the complete multi-agent workflow:
        Research → Write → Review
        """

        print(f"\n{'='*60}")
        print(f" STARTING MULTI-AGENT WORKFLOW FOR: {topic.upper()}")
        print(f"{'='*60}")

        workflow_start_time = datetime.now()


        self.log_workflow("STEP 1", "Initiating Research Phase")
        research_result = self.research_agent.process_task({'topic': topic})

        if not research_result.get('research_completed'):
            return {'error': 'Research phase failed'}

        print(f"\n Research Phase Complete: {research_result['sources_count']} sources found")


        self.log_workflow("STEP 2", "Initiating Writing Phase")
        writing_result = self.writing_agent.process_task({
            'research_data': research_result
        })

        if not writing_result.get('writing_completed'):
            return {'error': 'Writing phase failed'}

        print(f"\n Writing Phase Complete: {writing_result['word_count']} words written")


        self.log_workflow("STEP 3", "Initiating Review Phase")
        review_result = self.reviewing_agent.process_task({
            'writing_data': writing_result
        })

        if not review_result.get('review_completed'):
            return {'error': 'Review phase failed'}

        print(f"\n Review Phase Complete: {len(review_result['improvement_suggestions'])} improvements suggested")


        workflow_end_time = datetime.now()
        total_time = (workflow_end_time - workflow_start_time).total_seconds()

        final_result = {
            'coordinator': self.name,
            'topic': topic,
            'workflow_completed': True,
            'total_processing_time': f"{total_time:.2f} seconds",
            'research_data': research_result,
            'writing_data': writing_result,
            'review_data': review_result,
            'final_blog_post': review_result['improved_content'],
            'workflow_summary': {
                'research_sources': research_result['sources_count'],
                'original_word_count': writing_result['word_count'],
                'final_word_count': review_result['improved_word_count'],
                'improvements_made': len(review_result['improvement_suggestions'])
            }
        }

        self.log_workflow("WORKFLOW COMPLETE", f"Total time: {total_time:.2f}s")

        return final_result

    def display_workflow_summary(self, result: Dict[str, Any]):
        """Display a nice summary of the workflow results"""
        if result.get('error'):
            print(f" Workflow Error: {result['error']}")
            return

        print(f"\n{'='*60}")
        print(f" WORKFLOW SUMMARY FOR: {result['topic'].upper()}")
        print(f"{'='*60}")

        summary = result['workflow_summary']
        print(f" Research Sources Found: {summary['research_sources']}")
        print(f" Original Word Count: {summary['original_word_count']}")
        print(f" Final Word Count: {summary['final_word_count']}")
        print(f" Improvements Suggested: {summary['improvements_made']}")
        print(f" Total Processing Time: {result['total_processing_time']}")

        print(f"\n FINAL BLOG POST PREVIEW:")
        print("-" * 40)
        # Show first 200 characters of the final blog post
        preview = result['final_blog_post'][:200] + "..." if len(result['final_blog_post']) > 200 else result['final_blog_post']
        print(preview)
        print("-" * 40)


print(" Creating Multi-Agent Coordinator...")
coordinator = MultiAgentCoordinator()
print(" Multi-Agent Coordinator ready!")

 Creating Multi-Agent Coordinator...
 Multi-Agent Coordinator ready!


In [ ]:
print(" RUNNING COMPLETE MULTI-AGENT SYSTEM")
print("="*50)


user_topic = "The Future of Artificial Intelligence"

print(f" User Input Topic: {user_topic}")
print("\nStarting multi-agent workflow...\n")


workflow_result = coordinator.run_complete_workflow(user_topic)


coordinator.display_workflow_summary(workflow_result)


if workflow_result.get('review_data'):
    print(f"\n IMPROVEMENT SUGGESTIONS:")
    print("-" * 40)
    for i, suggestion in enumerate(workflow_result['review_data']['improvement_suggestions'], 1):
        print(f"{i}. {suggestion['category']}: {suggestion['suggestion']}")

 RUNNING COMPLETE MULTI-AGENT SYSTEM
 User Input Topic: The Future of Artificial Intelligence

Starting multi-agent workflow...


 STARTING MULTI-AGENT WORKFLOW FOR: THE FUTURE OF ARTIFICIAL INTELLIGENCE
[04:40:20]  Multi-Agent Coordinator: STEP 1 - Initiating Research Phase
 [04:40:20] Research Agent: Starting research on: The Future of Artificial Intelligence
 [04:40:20] Research Agent: Searching for information about: The Future of Artificial Intelligence
 [04:40:21] Research Agent: Found 5 relevant information pieces
 [04:40:21] Research Agent: Research completed! Found 5 key findings

 Research Phase Complete: 5 sources found
[04:40:21]  Multi-Agent Coordinator: STEP 2 - Initiating Writing Phase
 [04:40:21] Writing Agent: Starting to write blog post about: The Future of Artificial Intelligence
 [04:40:21] Writing Agent: Structuring content for: The Future of Artificial Intelligence
 [04:40:21] Writing Agent: Blog post completed! 202 words, 10 paragraphs

 Writing Phase Complete: 2

In [ ]:
def interactive_demo():
    """Interactive demo where users can input their own topics"""

    print(" INTERACTIVE MULTI-AGENT DEMO")
    print("=" * 40)
    print("Enter a topic you'd like our AI agents to research, write about, and review!")
    print("Examples: 'Climate Change', 'Space Exploration', 'Renewable Energy', etc.")
    print()


    demo_topics = [
        "Climate Change Solutions",
        "Space Exploration Technologies",
        "Renewable Energy Future",
        "Healthcare AI Applications",
        "Sustainable Transportation"
    ]

    print(" Demo Topics Available:")
    for i, topic in enumerate(demo_topics, 1):
        print(f"   {i}. {topic}")


    selected_topic = demo_topics[1]  # Space Exploration Technologies
    print(f"\n Running demo with topic: '{selected_topic}'")


    result = coordinator.run_complete_workflow(selected_topic)
    coordinator.display_workflow_summary(result)

    return result


demo_result = interactive_demo()

 INTERACTIVE MULTI-AGENT DEMO
Enter a topic you'd like our AI agents to research, write about, and review!
Examples: 'Climate Change', 'Space Exploration', 'Renewable Energy', etc.

 Demo Topics Available:
   1. Climate Change Solutions
   2. Space Exploration Technologies
   3. Renewable Energy Future
   4. Healthcare AI Applications
   5. Sustainable Transportation

 Running demo with topic: 'Space Exploration Technologies'

 STARTING MULTI-AGENT WORKFLOW FOR: SPACE EXPLORATION TECHNOLOGIES
[04:41:51]  Multi-Agent Coordinator: STEP 1 - Initiating Research Phase
 [04:41:51] Research Agent: Starting research on: Space Exploration Technologies
 [04:41:51] Research Agent: Searching for information about: Space Exploration Technologies
 [04:41:52] Research Agent: Found 5 relevant information pieces
 [04:41:52] Research Agent: Research completed! Found 5 key findings

 Research Phase Complete: 5 sources found
[04:41:52]  Multi-Agent Coordinator: STEP 2 - Initiating Writing Phase
 [04:41:52